# I. Recommended Musical Artists

When using the major music streaming services, there's one particular feature that I've found a bit lacking: the list of related artists on a band's page.

Take for example, the band Beartooth. These are their top 10 recommended artists from each of the four biggest music streaming services:

| Spotify              | Apple Music          | Amazon Music        | YouTube Music        |
| :---:                | :---:                | :---:               | :---:                |
| Bad Omens            | Wage War             | Of Mice and Men     | Of Mice and Men      |
| I Prevail            | Of Mice and Men      | Memphis May Fire    | A Day To Remember    |
| Hands Like Houses    | Ice Nine Kills       | I Prevail           | Crown The Empire     |
| Memphis May Fire     | Memphis May Fire     | A Day To Remember   | We Came As Romans    |
| The Word Alive       | I Prevail            | Motionless in White | Memphis May Fire     |
| Like Moths To Flames | Crown The Empire     | Crown The Empire    | Ice Nine Kills       |
| We Came As Romans    | Like Moths To Flames | We Came As Romans   | Wage War             |
| Crown The Empire     | The Amity Affliction | Throw The Fight     | While She Sleeps     |
| Ice Nine Kills       | Fit for a King       | Asking Alexandria   | Bring Me The Horizon |
| Miss May I           | Issues               | Hands Like Houses   | The Word Alive       |

Not a bad list of recommendations per se. Beartooth is a metalcore band that trends toward mainstream hard rock. And all of these recommended artists could also fit that description relatively well. But if I go down this list of artists, and look in turn at their recommended artists, I start seeing a lot of the same bands, all from the same genre.

By contrast, a couple of years ago, I went to see Beartooth live. That night, at that show, their opener was a group called Knocked Loose. Knocked Loose is a hardcore punk band that trends toward deathcore. And you will find them absolutely nowhere on any streaming service's list of artists similar to Beartooth. But at some point, someone, somewhere, decided that they would go well together on a bill. (And for what it's worth, I agree).

So what if instead, I could recommend related artists based on the touring history of the band in question, sorting by the number of shows played together and highlighting the top ten? 

# II. Using Tour History to Recommend Artists

[Setlist.fm](https://www.setlist.fm/) is a community-curated database of the songs that bands play at each of their shows. Importantly, it also contains other metadata like what venues they played at and on what dates. Therefore, for any given band, we can look up their tour history and compile a list of all of the different bands they have played with over the years. And fortunately, Setlist.fm has an API that we can use to crunch this data rather than doing it manually.

### Setlist.fm API configuration

Most websites with APIs feature some form of authentication in order to ensure that the users making requests of their servers are being good internet citizens (i.e. not making too many requests too often). Therefore, to use Setlist.fm's API, we'll need to [make an account and apply for an API KEY.](https://api.setlist.fm/docs/1.0/index.html)

Next, for our own security&mdash;so that we're not putting the raw key values directly into the code&mdash;we'll need to set our [API KEY](https://www.setlist.fm/settings/apps) up as an [environment variable](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#windows) which we can call.

Use the command below to determine your top-level conda environment directory.

In [1]:
! echo %CONDA_PREFIX%

From this location, make two files, one that sets the environment variables when the conda environment is activated and one that unsets them when it is deactivated.
```
%CONDA_PREFIX%\etc\conda\activate.d\env_vars.bat
%CONDA_PREFIX%\etc\conda\deactivate.d\env_vars.bat
```

To the activation script `%CONDA_PREFIX%\etc\conda\activate.d\env_vars.bat`, add the following line:
```
set SETLISTFM_API_KEY='YOUR-API-KEY'
```
And to the deactivation script `%CONDA_PREFIX%\etc\conda\deactivate.d\env_vars.bat`:
```
set SETLISTFM_API_KEY= 
```

### Setup

In [2]:
from SimilarBandFunctions import *

In [3]:
API_KEY_q = os.environ.get("SETLISTFM_API_KEY")
API_KEY = API_KEY_q.replace('\'','')
headers = {
    'x-api-key': API_KEY,
    'Accept': 'application/json'
}

### Download Artist Touring Data

In [4]:
Beartooth = Artist('Beartooth','6vwjIs0tbIiseJMR3pqwiL','98a1e0ab-35fa-40dd-b62c-9fda46fdb061')

In [5]:
tour_mates = FindTourMates(Beartooth.musicbrainz_id, headers)
tour_mates.set_band_cache_file('band_cache.txt')
tour_mates.run()

### Beartooth's top 10 artists based on tour history

In [6]:
tourmates_df = pd.read_json(tour_mates.band_database[Beartooth.musicbrainz_id]['similarBands'])
tourmates_df.sort_values('count', inplace = True, ascending = False)
print(Beartooth.name)
print(tourmates_df[0:10])

Beartooth
                                     mbid               name  count
14   5f4ad442-76d1-4cd3-8677-3cff7be4c8d4  Hands Like Houses    124
560  681b5ab2-b7dd-4dff-85e9-1e84503ad36a   Memphis May Fire     90
477  d89de379-665d-425c-b2e9-41b95d1edb36        Silverstein     89
69   6eaab7b4-f2cb-4c80-9b36-7e7d5c2fa8c5     Fit for a King     84
537  c41dd59f-d805-41df-9e0e-83ec0f9f468e          Neck Deep     80
294  d164d8e4-f505-4ed8-9345-c0bd1fa67ccc              Sylar     80
534  aafa70a4-2f06-4975-935e-b283fc87de7e       Blessthefall     78
22   1921c28c-ec61-4725-8e35-38dd656f7923          I Prevail     69
530  f0b1619b-6b76-4633-9a83-85b11a17ad98             Attila     68
503  8ed919fb-eaee-45a1-ba99-b3ede9ca5f1d    Pierce the Veil     65


# III. Quantifying Artist and Genre Diversity

Now that we have a list of recommended bands based on Beartooth's touring history, we want a way to quantitatively compare it to the lists given by the streaming services. Of the four largest streaming services, Spotify is the only one that has a freely available API, so that's the one we'll use moving forward. Since Spotify's recommended artists feature seems to need more variety, the quantitative metrics of interest will be artist diversity and genre diversity.

## Spotify API Setup

To make API requests of Spotify, we'll need to go through a similar setup process as we did for Setlist.fm. The first thing we'll need to do is create a [Spotify developer account](https://developer.spotify.com/dashboard/).

Spotify's API uses two authentication IDs, one for the individual user, and one for the app that is being used to make the requests. We will therefore need to create a spotify "app" that will service our API requests. From the Spotify developer dashboard, click on "Create An App" and give it any name and description you like.
From here, you will be taken to your app's page, where you can find the Client ID and Client Secret. These are the two authentication keys you'll need to make requests to Spotify.

To the same conda environment activation script from before (`%CONDA_PREFIX%\etc\conda\activate.d\env_vars.bat`), add the following lines:

```
set SPOTIFY_CLIENT_ID='YOUR-SPOTIFY-CLIENT-ID'
set SPOTIFY_SECRET='YOUR-SPOTIFY-SECRET'
```
And to the deactivation script (`%CONDA_PREFIX%\etc\conda\deactivate.d\env_vars.bat`):
```
set SPOTIFY_CLIENT_ID=
set SPOTIFY_SECRET=
```

Once you've set up your keys as environment variables, you can load them into the session:

In [7]:
SPOTIFY_CLIENT_ID_q = os.environ.get("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_ID = SPOTIFY_CLIENT_ID_q.replace('\'','')

SPOTIFY_SECRET_q = os.environ.get("SPOTIFY_SECRET")
SPOTIFY_SECRET = SPOTIFY_SECRET_q.replace('\'','')

To do the actual legwork of interfacing with Spotify and making requests for data, we'll leverage an existing library called spotipy.

In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = SPOTIFY_CLIENT_ID
secret = SPOTIFY_SECRET

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Pairing Tour Artists with their Spotify IDs
Setlist.fm and Spotify both have a unique ID code for each artist. In order to compare recommended artists from one to the other, we need to be able to cross-reference these IDs.

In [9]:
artistID_cross_reference = load_artists()

### Artist Diversity

To quantify artist diversity in a recommended list of 10 artists (either from tour history or directly from Spotify), we can use Spotify's API to check how many artists are provided when asking for a second round of recommendations.

So if the primary artist is Beartooth, we can ask Spotify for the top ten recommended artists for each artist on Beartooth's top ten list. Because some of the bands recommended will be repeats, this will make a good measure of artist diversity. We can apply this measure to lists of recommended bands both directly from Spotify and based on touring history.

In [10]:
def artist_diversity(artist_uri_list):
    spotify_related_artists = set()
    for i in range(0,10):
        artist1 = sp.artist(artist_uri_list[i])
        spotify_related_artists.add(artist1['name'])
        related_artists_2 = sp.artist_related_artists(artist_uri_list[i])
        for j in range(0,10):
            spotify_related_artists.add(related_artists_2['artists'][j]['name'])
    artist_diversity = len(spotify_related_artists)
    return(artist_diversity)

#### Spotify Artist Diversity

In [11]:
Beartooth_related_artists = sp.artist_related_artists(Beartooth.spotify_id)
related_uris = []
for i in range(0,10):
    related_uris.append(Beartooth_related_artists['artists'][i]['id'])
Beartooth.spotify_artist_diversity = artist_diversity(related_uris)
print(Beartooth.spotify_artist_diversity)

48


#### Touring Artist Diversity

In [12]:
Beartooth_uris = tour_mates.spotify_uris_from_database(Beartooth.musicbrainz_id,artistID_cross_reference)
Beartooth.touring_artist_diversity = artist_diversity(Beartooth_uris)
print(Beartooth.touring_artist_diversity)

85


### Genre Diversity

We can also take a look at how this approach impacts genre diversity. Spotify lists multiple genres for each band, and a different number for each. We want all of a band's genres to be represented, but we don't want to give a band more credit for having many increasingly niche genres on their profile. Therefore, we'll give each band a single genre point to be divided up by however many genres they have. We'll also cap the amount each genre can receive at maximum of one point to place the emphasis on diversity. Adding up these points for a list of 10 recommended artists gives us a score for genre diversity.

In [13]:
def genre_diversity(spotify_artist_list):
    genre_dict = {}
    for i in range(0,len(spotify_artist_list)):
        genre_list = spotify_artist_list[i]['genres']
        genre_count = len(genre_list)
        for genre in genre_list:
            genre_dict[genre] = genre_dict.get(genre, 0) + 1/genre_count # Each band only gets one point worth of genre representation
    genre_score_list = genre_dict.values()
    genre_score_list2 = [] 
    for score in genre_score_list:
        genre_score_list2.append(min(1,score)) # Genres can only be represented a maximum of once
    genre_score = sum(genre_score_list2)
    return(genre_dict,genre_score)

#### Spotify Genre Diversity

In [14]:
result = sp.artist_related_artists(Beartooth.spotify_id)
spotify_artist_list = result['artists'][0:10]
spotify_genre_dict,spotify_genre_score = genre_diversity(spotify_artist_list)
spotify_genre_score
Beartooth.spotify_genre_diversity = spotify_genre_score
print(Beartooth.spotify_genre_diversity)

7.4833333333333325


#### Touring Genre Diversity

In [15]:
tour_artist_list= []
for band in Beartooth_uris:
    tour_artist_list.append(sp.artist(band))
tour_genres, tour_genre_score = genre_diversity(tour_artist_list)
tour_genre_score
Beartooth.touring_genre_diversity = tour_genre_score
print(Beartooth.touring_genre_diversity)

8.96190476190476


#### Beartooth Metrics

In [16]:
pd.options.display.float_format = "{:,.1f}".format

columns = ['Spotify Artist Diversity',
           'Touring Artist Diversity',
           'Spotify Genre Diversity',
           'Touring Genre Diversity']
bands = ['Beartooth']
df = pd.DataFrame(columns=columns,
                index=bands)

current_artist = Beartooth
df.loc[current_artist.name] = pd.Series({'Spotify Artist Diversity': current_artist.spotify_artist_diversity,
           'Touring Artist Diversity': current_artist.touring_artist_diversity,
           'Spotify Genre Diversity': current_artist.spotify_genre_diversity,
           'Touring Genre Diversity': current_artist.touring_genre_diversity})
df

,Spotify Artist Diversity,Touring Artist Diversity,Spotify Genre Diversity,Touring Genre Diversity
Beartooth,48.0,85.0,7.5,9.0


# IV. Validation with Additional Artists

Recommending artists based on touring history worked well for Beartooth. It increased both the metrics of artist and genre diversity. The question now becomes whether the method is generalizable. So let's try it again for 10 different artists adding some variety to era, popularity, and genre (while still staying roughly within the realm of rock music).

In [17]:
PapaRoach       = Artist('Papa Roach'       ,'4RddZ3iHvSpGV4dvATac9X','c5eb9407-caeb-4303-b383-6929aa94021c')
Bearings        = Artist('Bearings'         ,'0qpDBxRgLp6g0k2esJlUDn','5a19c7e6-b435-45f4-b1de-2db9b3271cc5')
KnockedLoose    = Artist('Knocked Loose'    ,'4qrHkx5cgWIslciLXUMrYw','9ca10859-49e2-44e3-b3d6-04c535207bc2')
Chevelle        = Artist('Chevelle'         ,'56dO9zeHKuU5Gvfc2kxHNw','8456e9f7-debf-4579-a86c-33a325a35d2d')
MotleyCrue      = Artist('Motley Crue'      ,'0cc6vw3VN8YlIcvr1v7tBL','26f07661-e115-471d-a930-206f5c89d17c')
Periphery       = Artist('Periphery'        ,'6d24kC5fxHFOSEAmjQPPhc','a0cef17a-4574-44f4-9f97-fd068615dac6')
Counterparts    = Artist('Counterparts'     ,'5LyRnL0rysObxDRxzSfV1z','4b0dd5e7-c795-42bd-8311-bc9f71fabd0a')
HollywoodUndead = Artist('Hollywood Undead' ,'0CEFCo8288kQU7mJi25s6E','321fdfbb-426b-43f7-8295-fa9aca6348d9')
MachineGunKelly = Artist('Machine Gun Kelly','6TIYQ3jFPwQSRmorSezPxX','f6af669a-56ea-448a-a044-de76181ada33')
Crosses         = Artist('Crosses'          ,'3gPZCcrc8KG2RuVl3rtbQ2','7a10215e-b32f-4b77-b9cc-d90531a3968f')

artist_list = []
artist_list.append(PapaRoach)
artist_list.append(Bearings)
artist_list.append(KnockedLoose)
artist_list.append(Chevelle)
artist_list.append(MotleyCrue)
artist_list.append(Periphery)
artist_list.append(Counterparts)
artist_list.append(HollywoodUndead)
artist_list.append(MachineGunKelly)
artist_list.append(Crosses)

## Download Data

In [18]:
related_artist_uris = []
for a in range(0,len(artist_list)):
    artist_ID = artist_list[a].musicbrainz_id
    tour_mates = FindTourMates(artist_ID, headers)
    print(tour_mates.primary_band_mbid)
    tour_mates.set_band_cache_file('band_cache.txt')
    tour_mates.run()
    artist_ID = artist_list[a].musicbrainz_id
    spotify_uris = tour_mates.spotify_uris_from_database(artist_ID,artistID_cross_reference)
    related_artist_uris.append(spotify_uris)

c5eb9407-caeb-4303-b383-6929aa94021c
5a19c7e6-b435-45f4-b1de-2db9b3271cc5
9ca10859-49e2-44e3-b3d6-04c535207bc2
8456e9f7-debf-4579-a86c-33a325a35d2d
26f07661-e115-471d-a930-206f5c89d17c
a0cef17a-4574-44f4-9f97-fd068615dac6
4b0dd5e7-c795-42bd-8311-bc9f71fabd0a
321fdfbb-426b-43f7-8295-fa9aca6348d9
f6af669a-56ea-448a-a044-de76181ada33
7a10215e-b32f-4b77-b9cc-d90531a3968f


### Display Top Ten Recommended Artists for each Artist

In [19]:
for a in range(0,len(artist_list)):
    tour_mates_df = pd.read_json(tour_mates.band_database[artist_list[a].musicbrainz_id]['similarBands'])
    tour_mates_df.sort_values('count', inplace = True, ascending = False)
    print(artist_list[a].name)
    print(tour_mates_df[0:10])
    print('\n')

Papa Roach
                                      mbid               name  count
429   822e92ef-72ea-42e0-9af1-b987816b487a         Buckcherry    150
24    adc0f033-95c2-4e0b-87bc-c23ed3f26ce6          Shinedown     95
1486  19516266-e5d9-4774-b749-812bb76a6559         (həd) p.e.     93
176   29266b3d-b5ae-4d09-b721-326246adf68f     In This Moment     86
25    9d2fde91-4633-430d-87f0-2b9bbb7fa451  Asking Alexandria     80
631   f10177e4-b7f7-4e63-92b6-c805c6cc54d6           Nonpoint     79
175   4bb4e4e4-5f66-4509-98af-62dbb90c45c5          Disturbed     70
71    a466c2a2-6517-42fb-a160-1087c3bafd9f           Slipknot     65
443   fbcd7b29-455f-49e6-9c4f-8249d20a055e            Seether     64
1030  39b22a9e-59dd-412b-ac3c-0725c807c72b             Hinder     62


Bearings
                                    mbid              name  count
1   687823ac-a3b3-495c-bb20-68bce137c77d       Sleep On It     18
0   a4dbddc8-a08d-424e-b0b7-69b423681190  Between You & Me     12
11  e2e9df76-a950-4b8

### Spotify Artist Diversity

In [20]:
for a in range(0,len(artist_list)):
    current_artist = artist_list[a]
    current_artist_sp_id = current_artist.spotify_id
    related_artists_1 = sp.artist_related_artists(current_artist_sp_id)
    related_uris = []
    for i in range(0,10):
        related_uris.append(related_artists_1['artists'][i]['id'])
    artist_list[a].spotify_artist_diversity = artist_diversity(related_uris)

### Touring Artist Diversity

In [21]:
for a in range(0,len(artist_list)):
    if a < len(related_artist_uris):
        spotify_uris = related_artist_uris[a]
        artist_list[a].touring_artist_diversity = artist_diversity(spotify_uris)

### Spotify Genre Diversity

In [22]:
for a in range(0,len(artist_list)):
    result = sp.artist_related_artists(artist_list[a].spotify_id)
    spotify_artist_list = result['artists'][0:10]
    spotify_genre_dict,spotify_genre_score = genre_diversity(spotify_artist_list)
    artist_list[a].spotify_genre_diversity = spotify_genre_score

### Touring Genre Diversity

In [23]:
for a in range(0,len(artist_list)):
    if a < len(related_artist_uris):
        spotify_uris = related_artist_uris[a]
        tour_artist_list= []
        for band in spotify_uris:
            tour_artist_list.append(sp.artist(band))
        tour_genres, tour_genre_score = genre_diversity(tour_artist_list)
        artist_list[a].touring_genre_diversity = tour_genre_score

## Display Artist/Genre Diversity Metrics

In [24]:
columns = ['Spotify Artist Diversity',
           'Touring Artist Diversity',
           'Spotify Genre Diversity',
           'Touring Genre Diversity']
bands = ['Papa Roach',
         'Bearings',
         'Knocked Loose',
         'Chevelle',
         'Motley Crue',
         'Periphery',
         'Counterparts',
         'Hollywood Undead',
         'Machine Gun Kelly',
         'Crosses']
df = pd.DataFrame(columns=columns,
                index=bands)

In [25]:
for a in range(0,len(artist_list)):
    current_artist = artist_list[a]
    df.loc[current_artist.name] = pd.Series({'Spotify Artist Diversity': current_artist.spotify_artist_diversity,
               'Touring Artist Diversity': current_artist.touring_artist_diversity,
               'Spotify Genre Diversity': current_artist.spotify_genre_diversity,
               'Touring Genre Diversity': current_artist.touring_genre_diversity})

In [26]:
df

,Spotify Artist Diversity,Touring Artist Diversity,Spotify Genre Diversity,Touring Genre Diversity
Papa Roach,38.0,76.0,6.2,7.7
Bearings,35.0,57.0,7.0,7.0
Knocked Loose,29.0,102.0,5.0,9.7
Chevelle,34.0,58.0,6.4,6.9
Motley Crue,38.0,71.0,5.9,7.4
Periphery,32.0,80.0,6.3,9.5
Counterparts,52.0,99.0,7.0,8.9
Hollywood Undead,59.0,97.0,9.3,9.9
Machine Gun Kelly,80.0,105.0,8.9,9.8
Crosses,61.0,106.0,7.6,10.0


# V. Discussion

By recommending similar artists based on touring history, we were able to increase the diversity and the genre diversity of the artists recommended over those directly presented by Spotify. Tour-based recommendations offer more artists across a broader spectrum of styles. But are these recommendations better from the perspective of the end user? Well, it depends what your point of comparison is.

Relative to the related artists feature specifically? I'd say so. But relative to Spotify's overall approach to recommending new music? Definitely not. Spotify is a deeply data-driven company that uses a large multi-faceted algorithm to make recommendations to users. [As of 2015, three major features of the algorithm](https://www.slideshare.net/MrChrisJohnson/from-idea-to-execution-spotifys-discover-weekly/31-1_0_0_0_1) were Implicit Matrix Factorization for crunching user preference and behavioral data, Natural Language Processing to extract cultural and relational metadata from news, blogs, and playlists, and Deep Learning applied to raw audio data to classify songs and artists. I have by no means discovered a magic bullet capable of upending Spotify's cutting-edge algorithm. What I have discovered is a small blindspot.

Any algorithm is liable to develop biases, particularly as it expands to include many diverse features and sources of information. In this case, the bias in question is too narrow a view of which artists and genres are related to which within the context of of the related artists feature. From Spotify's perspective, a similar result could plausibly be reached by instead of simply presenting the top 10 related artists, probabalistically presenting 10 related artists from the top 200 weighted by their position on the list. The relative positional weights could then be tuned to balance the diversity and relevance of the recommendations overall.

The other advantage of touring-based recommendations is their interpretability. Large algorithms can frequently appear opaque to humans observers, especially if they combine many different sources of information from different domains. Putting together a tour, on the other hand, is a direct function of the music industry, involving artists and other professionals like managers, agents, labels, and venue owners. By deciding which bands should go together on a tour, they contribute to the social construction of which artists are similar to which and what genres they fall into. 

There are several further developments that would be necessary to take this method from proof-of-concept to a tool that an end-user could feasibly use to discover new artists. The limited download throughput of Setlist.fm's API reflects the fact that they are simply not as large and well-resourced as a company like Spotify. As this project is non-commercial, I decided to stick to the recommended download rate and daily cap. According to their documentation, Setlist.fm may choose to increase the download rate by 8x and the daily cap by 35x by request. However, downloading such a large quantity of data piecemeal through the API is not ideal to begin with. Determining which other artists also performed at the venue and on the date of a given setlist is responsible for the majority of the API requests performed here. [Individual setlist pages](https://www.setlist.fm/setlist/beartooth/2020/o2-academy-bristol-bristol-england-1399bda1.html) will sometimes contain this information, but it is not directly available within a setlist object downloaded from the API. The addition of such a field could greatly improve the performance of the functions that compile the list of tourmates.

The other improvement that would be essential is [automatically cross-referencing Spotify and Musicbrainz (Setlist.fm) IDs](https://github.com/metabrainz/mbspotify). As it stands, the method is capable of providing a list of recommendations based on touring history requiring only the Musicbrainz ID of the band in question. However, in order to apply any of the metrics pulled from the Spotify API, the user must also look up and manually enter the IDs for all of the recommended bands. Automatic cross-referencing would remove this burden from the end-user, making the tool much more practical.